In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('csv/K15T_K16_K17_.csv')
K15T = pd.read_csv('../sql/K15T_UNEMPLOYED_RESIDENTS_BY_PREVIOUS_OCCUPATION_AND_SEX.csv')
K16 = pd.read_csv('../sql/K16_UNEMPLOYED_RESIDENT_PMETs_AND_NON-PMETs__BY_AGE.csv')
K17 = pd.read_csv('../sql/K17_LONG-TERM_UNEMPLOYED_RESIDENT_BY_PMET_STATUS_AND_AGE.csv')

In [4]:
K15T.fillna(0, inplace = True)
K15T_drop = K15T[K15T['Occupation'] != 'others who have never worked before']
year_cols = [str(year) for year in range(2014, 2025)]
long_K15T = K15T_drop.melt(id_vars=['Occupation', 'Gender'], value_vars=year_cols, var_name="Year", value_name="Unemployed_Number")
long_K15T['Year'] = long_K15T['Year'].astype(int)
long_K15T.head()
long_K15T.head()

,Occupation,Gender,Year,Unemployed_Number
0,Managers & Administrators (Including Working P...,Male,2014,6.1
1,Professionals,Male,2014,4.9
2,Associate Professionals & Technicians,Male,2014,6.3
3,Clerical Support Workers,Male,2014,4.3
4,Service & Sales Workers,Male,2014,6.5


In [5]:
K16.fillna(0, inplace = True)
K16_drop = K16[K16['Age'] != 'Undefined']
year_cols = [str(year) for year in range(2014, 2025)]
long_K16 = K16_drop.melt(id_vars=['Pmet_Status', 'Age'], value_vars=year_cols, var_name="Year", value_name="Unemployed_Number")
long_K16['Year'] = long_K16['Year'].astype(int)
print(long_K16.shape)
long_K16.head()

(88, 4)


,Pmet_Status,Age,Year,Unemployed_Number
0,PMETs,15 - 29,2014,8.3
1,PMETs,30 - 39,2014,8.6
2,PMETs,40 - 49,2014,8.7
3,PMETs,50 & Over,2014,6.9
4,Non-PMETs,15 - 29,2014,15.2


In [6]:
K17.fillna(0, inplace = True)
year_cols = [str(year) for year in range(2014, 2025)]
long_K17 = K17.melt(id_vars=['Pmet_Status', 'Age','Status'], value_vars=year_cols, var_name="Year", value_name="Unemployed_Number")
long_K17['Year'] = long_K17['Year'].astype(int)
print(long_K17.shape)
long_K17.head()

(176, 5)


,Pmet_Status,Age,Status,Year,Unemployed_Number
0,PMETs,15 - 29,Long-term,2014,0.7
1,PMETs,30 - 39,Long-term,2014,1.4
2,PMETs,40 - 49,Long-term,2014,2.5
3,PMETs,50 & Over,Long-term,2014,2.4
4,Non-PMETs,15 - 29,Long-term,2014,0.9


In [7]:
### Prove by Reproducing K15T

K15T_df = df.groupby(['Occupation','Year','Gender']).sum().reset_index()
# K15T_df[(K15T_df['Gender']=='Male') & (K15T_df['Occupation']=='Associate Professionals & Technicians')]

Occupation_list = K15T_df['Occupation'].unique()
Year_list = K15T_df['Year'].unique()
Gender_list = K15T_df['Gender'].unique()

# count_discrepancy = 0 
discrepancy_records = []

for occp in Occupation_list:
    for year in Year_list:
        for gender in Gender_list:
            count_vals = K15T_df.loc[(K15T_df['Occupation'] == occp) & (K15T_df['Year'] == year) & (K15T_df['Gender'] == gender), 'Count (Thousands)'].reset_index(drop=True)
            unemployed_vals = long_K15T.loc[(long_K15T['Occupation'] == occp) & (long_K15T['Year'] == year) & (long_K15T['Gender'] == gender), 'Unemployed_Number'].reset_index(drop=True)
            if (abs(count_vals - unemployed_vals) > 0.15).any():
                # print(gender , year, occp)
                # print(count_vals[0], unemployed_vals[0])
                # count_discrepancy += 1
                discrepancy_records.append({
                    'Gender': gender,
                    'Year': year,
                    'Occupation': occp,
                    'Count_Thousands': count_vals.iloc[0],
                    'Unemployed_Number': unemployed_vals.iloc[0],
                    'Difference': count_vals.iloc[0] - unemployed_vals.iloc[0]
                })

# print(count_discrepancy)
discrepancies_df = pd.DataFrame(discrepancy_records)

discrepancies_df


""


In [ ]:
### Prove by Reproducing K16

K16_df = df.groupby(['Pmet_Status', 'Age_Group','Year']).sum().reset_index()

Pmet_list = K16_df['Pmet_Status'].unique()
Year_list = K16_df['Year'].unique()
Age_list = K16_df['Age_Group'].unique()


# count_discrepancy = 0 
discrepancy_records = []

for pmet in Pmet_list:
    for year in Year_list:
        for age in Age_list:
            count_vals = K16_df.loc[(K16_df['Pmet_Status'] == pmet) & (K16_df['Year'] == year) & (K16_df['Age_Group'] == age), 'Count (Thousands)'].reset_index(drop=True)
            unemployed_vals = long_K16.loc[(long_K16['Pmet_Status'] == pmet) & (long_K16['Year'] == year) & (long_K16['Age'] == age), 'Unemployed_Number'].reset_index(drop=True)
            if (abs(count_vals - unemployed_vals) > 0.01).any():
                discrepancy_records.append({
                    'Pmet_Status': pmet,
                    'Year': year,
                    'Age': age,
                    'Count_Thousands': count_vals.iloc[0],
                    'Unemployed_Number': unemployed_vals.iloc[0],
                    'Difference': count_vals.iloc[0] - unemployed_vals.iloc[0]
                })

# print(count_discrepancy)
discrepancies_df = pd.DataFrame(discrepancy_records)

discrepancies_df

""


In [ ]:
### Prove by Reproducing K17

# print(long_K17.head(2))

K17_df = df.groupby(['Pmet_Status', 'Age_Group','Year','Type']).sum().reset_index()
# print(K17_df.head(2))

Pmet_list = K17_df['Pmet_Status'].unique()
Year_list = K17_df['Year'].unique()
Age_list = K17_df['Age_Group'].unique()
Type_list = K17_df['Type'].unique()

# count_discrepancy = 0 
discrepancy_records = []

for pmet in Pmet_list:
    for year in Year_list:
        for age in Age_list:
            for type in Type_list:
                count_vals = K17_df.loc[(K17_df['Pmet_Status'] == pmet) & (K17_df['Year'] == year) & (K17_df['Age_Group'] == age) & (K17_df['Type'] == type), 'Count (Thousands)'].reset_index(drop=True)
                unemployed_vals = long_K17.loc[(long_K17['Pmet_Status'] == pmet) & (long_K17['Year'] == year) & (long_K17['Age'] == age) & (long_K17['Status'] == type), 'Unemployed_Number'].reset_index(drop=True)
                if (abs(count_vals - unemployed_vals) > 0.001).any():
                    discrepancy_records.append({
                        'Pmet_Status': pmet,
                        'Year': year,
                        'Age': age,
                        'Type': type,
                        'Count_Thousands': count_vals.iloc[0],
                        'Unemployed_Number': unemployed_vals.iloc[0],
                        'Difference': count_vals.iloc[0] - unemployed_vals.iloc[0]
                    })

# print(count_discrepancy)
discrepancies_df = pd.DataFrame(discrepancy_records)

discrepancies_df

""
